In [1]:
from datasets import load_dataset, load_from_disk
from src.utils import CODEBASE_DIR
from src.preprocessing import create_audio_dataset
from src.speaker_verification import compute_similarity_score, compute_pairwise_similarity
import torch

/om2/user/azain/anaconda/envs/voice-anon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# dataset_with_embeddings = process_data_to_embeddings(f"{CODEBASE_DIR}/tmp/LibriTTS-processed-with-embeddings-small-shuffled")
# dataset_with_embeddings.save_to_disk(embeddings_dataset_path)
dataset_with_embeddings = load_from_disk(f'{CODEBASE_DIR}/tmp/LibriTTS-processed-with-embeddings-small-shuffled') # TODO load from tmp since cant load from hugging face atm
dataset_with_embeddings

Dataset({
    features: ['audio', 'transcript', 'speaker_id', 'embeddings'],
    num_rows: 100
})

In [21]:
dataset_with_embeddings[0].keys()

dict_keys(['audio', 'transcript', 'speaker_id', 'embeddings'])

In [22]:
embedding1 = torch.tensor(dataset_with_embeddings[0]['embeddings']).squeeze()
embedding2 = torch.tensor(dataset_with_embeddings[1]['embeddings']).squeeze()
embedding1.shape

torch.Size([192])

In [25]:
compute_similarity_score(embedding1, embedding1)

1.0

In [24]:
compute_similarity_score(embedding1, embedding2)

0.10053510963916779

In [26]:
# similarities = compute_pairwise_similarity(dataset_with_embeddings)
# similarities.save_to_disk(f"{CODEBASE_DIR}/tmp/LibriTTS-processed-with-embeddings-small-similarities")
similarities = load_from_disk(f"{CODEBASE_DIR}/tmp/LibriTTS-processed-with-embeddings-small-shuffled-similarities")
similarities

Dataset({
    features: ['speaker_id1', 'speaker_id2', 'embedding1', 'embedding2', 'similarity_score'],
    num_rows: 4950
})

In [29]:
similarities[0]

{'speaker_id1': '6313',
 'speaker_id2': '2412',
 'embedding1': [-15.22639274597168,
  -5.73972225189209,
  1.3488572835922241,
  -5.186030387878418,
  -6.6654767990112305,
  17.284717559814453,
  17.289936065673828,
  26.483503341674805,
  -0.21859902143478394,
  -20.249502182006836,
  -28.198123931884766,
  -4.249917984008789,
  9.331607818603516,
  -15.091987609863281,
  11.937324523925781,
  -6.758203506469727,
  -3.562268018722534,
  -12.571390151977539,
  0.22512605786323547,
  -4.901829719543457,
  -2.2403721809387207,
  -16.171171188354492,
  -18.98280906677246,
  17.430042266845703,
  -12.874014854431152,
  -5.8433003425598145,
  -3.4856503009796143,
  15.992561340332031,
  9.072793006896973,
  -1.5643538236618042,
  -17.33502197265625,
  -18.037723541259766,
  3.5076069831848145,
  -15.886507987976074,
  25.885042190551758,
  -22.935195922851562,
  3.0626418590545654,
  24.656280517578125,
  10.388062477111816,
  8.619298934936523,
  -23.429454803466797,
  -19.25633430480957,


In [41]:
low_similarity_same_speakers = 0
high_similarity_different_speakers = 0

threshold = 0.6
for item in similarities:
    if item['similarity_score'] > threshold and item['speaker_id1']!=item['speaker_id2']:
        high_similarity_different_speakers += 1
    if item['similarity_score'] < threshold and item['speaker_id1']==item['speaker_id2']:
        low_similarity_same_speakers += 1
print(f"{high_similarity_different_speakers=}, {low_similarity_same_speakers=}")

'high_similarity_different_speakers=0, low_similarity_same_speakers=83'